In [ ]:
import os
import json

from PIL import Image

import matplotlib

import matplotlib.pyplot as plt
import matplotlib.patches as patches

import random

import cv2

import glob


In [ ]:
if not os.path.exists("./garbage"):
    os.mkdir("./garbage")

In [ ]:
IMAGES_TRAIN = "../../../text-to-image/text-to-image pytorch implementation/datasets/COCO/images/train2017/"
IMAGES_VAL = "../../../text-to-image/text-to-image pytorch implementation/datasets/COCO/images/val2017/"
INSTAN_PATH_VAL = "../GCN2LY/data/datasets/COCO/annotations/instances_val2014.json"

GRAPHS = "../RNN2LY/data/datasets/AMR2014train-dev-test/GraphTest.json"

# Set the 3 epochs to compare
EVALUATE_EPOCHS = [
    {"path": "../GCN2LY/evaluator_output/objgan/execution0.json", "resize":False, "wh":True, "center":False, "uqcap":False},
    {"path": "../GCN2LY/evaluator_output/1/TESTINGepoch28.json", "resize":True, "wh":True, "center":True, "uqcap":False},
    {"path": "../RNN2LY/evaluator_output/1/TESTINGepoch28.json", "resize":True, "wh":True, "center":True, "uqcap":False},
]

In [ ]:
def draw_bounding_boxes(image_id, V, results, categories, resize=False, size=(256, 256), wh=True, center=False):
    try:
        picture = Image.open(IMAGES_TRAIN + "0" * (12 - len(str(image_id))) + str(image_id) + ".jpg")
    except:
        picture = Image.open(IMAGES_VAL + "0" * (12 - len(str(image_id))) + str(image_id) + ".jpg")
        
    fig, ax = plt.subplots(1, figsize=(15,15))
    ax.imshow(picture.resize(size), origin="upper")
    
    labels = []
    color = [
             "#FF0000","#00FF00", "#0000FF", "#FFFF00", "#00FFFF", "#FF00FF", "#0099FF", "#EB70AA", "#F0D58C", "#F4A460", "#FFD700", "#6495ED", "#000000", "#FFFFFF", 
             "#FF0000","#00FF00", "#0000FF", "#FFFF00", "#00FFFF", "#FF00FF", "#0099FF", "#EB70AA", "#F0D58C", "#F4A460", "#FFD700", "#6495ED", "#000000", "#FFFFFF",
             "#FF0000","#00FF00", "#0000FF", "#FFFF00", "#00FFFF", "#FF00FF", "#0099FF", "#EB70AA", "#F0D58C", "#F4A460", "#FFD700", "#6495ED", "#000000", "#FFFFFF",
             "#FF0000","#00FF00", "#0000FF", "#FFFF00", "#00FFFF", "#FF00FF", "#0099FF", "#EB70AA", "#F0D58C", "#F4A460", "#FFD700", "#6495ED", "#000000", "#FFFFFF",
             "#FF0000","#00FF00", "#0000FF", "#FFFF00", "#00FFFF", "#FF00FF", "#0099FF", "#EB70AA", "#F0D58C", "#F4A460", "#FFD700", "#6495ED", "#000000", "#FFFFFF",
             "#FF0000","#00FF00", "#0000FF", "#FFFF00", "#00FFFF", "#FF00FF", "#0099FF", "#EB70AA", "#F0D58C", "#F4A460", "#FFD700", "#6495ED", "#000000", "#FFFFFF",
            ]
    for k, result in enumerate(results):
        bbox, ls = result[:4], int(result[4])
        x, y, width, height = bbox
        
        if center:
            x = x - width / 2
            y = y - height / 2
            
        elif not wh:
            width = width - x
            height = height - y
            
        if resize:
            x*=size[0]
            y*=size[0]
            width*=size[0]
            height*=size[0]
        rect = patches.Rectangle((x, y),width, height, linewidth=3, edgecolor=color[k], facecolor='none')
        ax.add_patch(rect)
        labels.append(categories[ls])
        
    plt.legend(labels)
    # plt.show()
    plt.savefig("./garbage/"+str(image_id)+"-" + str(V))
    plt.close()

read gt

In [ ]:
with open(INSTAN_PATH_VAL, "r") as json_file:
    coco = json.load(json_file)

In [ ]:
image_id_to_size = {}
for object_data in coco['images']:
    image_id = str(object_data['id'])
    image_id_to_size[image_id] = (object_data['width'], object_data['height'])

In [ ]:
objects = {}
for object_data in coco['annotations']:
    image_id = str(object_data['image_id'])
    bbox = object_data['bbox']
    WW, HH = image_id_to_size[image_id]
    bbox[0] = bbox[0] / WW * 256
    bbox[1] = bbox[1] / HH * 256
    bbox[2] = bbox[2] / WW * 256
    bbox[3] = bbox[3] / HH * 256
    
    if image_id in objects:
        objects[image_id].append(bbox+[object_data['category_id']])
    else:
        objects[image_id] = [bbox+[object_data['category_id']]] 

In [ ]:
category_to_name = {}
for object_data in coco['categories']:
    category_to_name[object_data['id']] = object_data['name']

In [ ]:
with open(GRAPHS, "r") as json_file:
    graphs = json.load(json_file)

In [ ]:
def generate_picture(key, resize, wh, center, V, data, uqcap):
    if uqcap:
        key = key.split("-")[0]
    image_id = key.split("-")[0]
    draw_bounding_boxes(image_id, V, data[key], category_to_name, resize=resize, wh=wh, center=center)
    draw_bounding_boxes(image_id, "gt", objects[image_id], category_to_name)

select some picture to compare

In [ ]:
pictures_to_check1 = ['477542-3', '519605-0', '379667-4', '452784-3', '420472-2', '304657-1', '539775-1', '536517-0', '486254-1', '302520-2', '539777-1', '325473-0', '87561-3', '302388-1', '513064-2', '392364-1', '212573-0', '112378-3', '220670-4', '193889-0', '421745-1', '281317-0', '100001-2', '19783-4', '551167-2', '190160-3', '251042-1', '472295-2', '45053-2', '469687-0', '65358-1', '194097-0', '195559-2', '412247-1', '27530-1', '440043-2', '577451-1', '554620-4', '490199-1', '251042-1', '550714-3', '433892-1', '370208-0', '163253-0', '226058-4', '69536-3', '367018-4', '493442-1', '317763-4', '26982-4', '520237-0', '21688-2', '267910-4', '206806-2', '138871-3', '109992-2', '466635-2', '413734-3', '172850-0', '10986-4', '572734-3', '515982-0', '149406-4', '161642-4', '79407-0', '280810-3', '34826-3', '52746-4', '213809-0', '319487-2', '43611-3', '257657-0', '309655-3', '548159-3', '164899-0', '377486-3', '39628-4', '48273-4', '142574-4', '379014-2', '241319-1', '442175-0', '6417-2', '249273-0', '394157-1', '141770-2', '18531-2', '363058-1', '172513-0', '242103-3', '520478-2', '93040-0', '263613-0', '229000-0', '122302-3', '324155-0', '100215-0', '1799-3', '64796-2', '33854-4', '12543-4', '23411-2', '327029-2', '501762-1', '527979-1', '107853-3', '138095-1', '170227-2', '442345-2', '498082-1']
print(len(pictures_to_check1))

In [ ]:
i = 0

In [ ]:
for file in EVALUATE_EPOCHS:
    print("Checking", file['path'])
    with open(file['path'], "r") as json_file:
        data = json.load(json_file)
        for key in pictures_to_check1:
            try:
                generate_picture(key, file['resize'], file['wh'], file['center'], i, data, file['uqcap'])
            except:
                print(key, file['path'])

    i += 1

concatenate pictures

In [ ]:
def generate_picture_full(id, caption, orig, pred1, pred2, pred3):
    font = cv2.FONT_HERSHEY_SIMPLEX
    im1 = cv2.imread(orig)
    
    cv2.putText(im1, "ORIGINAL", (500, 60), font, 1,(0,0,0),2)
    im2 = cv2.imread(pred1)
    
    cv2.putText(im2, "PREDICTION-1", (500, 60), font, 1,(0,0,0),2)
    
    im3 = cv2.imread(pred2)
    cv2.putText(im3, "PREDICTION-2", (500, 60), font, 1,(0,0,0),2)
    
    im4 = cv2.imread(pred3)
    cv2.putText(im4, "PREDICTION-3", (500, 60), font, 1,(0,0,0),2)
    
    im_h = cv2.hconcat([im1, im2])
    im_v = cv2.hconcat([im3, im4])
    imv_v_final = cv2.vconcat([im_h, im_v])
    
    cv2.putText(imv_v_final, caption, (750, 1000), font, 1,(0,0,0),2)
    cv2.imwrite("./pictures_output/"+ str(id) + ".png", imv_v_final)

In [ ]:
if not os.path.exists("./pictures_output"):
    os.mkdir("./pictures_output")

In [ ]:
for i in range(len(pictures_to_check1)):
    key, cap = pictures_to_check1[i].split("-")
    p1, p2, p3, p4 = "./garbage/" + key +"-gt.png", "./garbage/" + key +"-0.png", "./garbage/" + key +"-1.png", "./garbage/" + key +"-2.png"        
    if not os.path.exists(p1):
        continue
    if not os.path.exists(p2):
        p2 = p1
    if not os.path.exists(p3):
        p3 = p1
    if not os.path.exists(p4): 
        p4 = p1
    caption = graphs[key]['graphs'][int(cap)]['caption'][:-1]
    # triples = graphs[image_id]['graphs'][int(cap)]['triples']
    generate_picture_full(key, caption, p1, p2, p3, p4)
    
    
    